In [92]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import KNeighborsClassifier

In [93]:
warnings.simplefilter('ignore')

In [94]:
data_raw = pd.read_csv('exo-planets.csv')
data_raw.shape

(5657, 3198)

In [104]:
data_raw['exo'].value_counts()

0    5615
1      42
Name: exo, dtype: int64

In [95]:
data_raw.head()

,exo,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,1,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,1,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,1,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,1,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,1,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54


---

In [96]:
data = data_raw.drop('exo', axis=1)
data = ((data-data.mean())/data.std())*10
data['exo'] = data_raw['exo']

In [97]:
test = data.loc[:89]
train = data.loc[90:].drop('exo', axis=1)

In [98]:
mod = LocalOutlierFactor(contamination=0.18, n_neighbors=25, novelty=False)
preds = mod.fit_predict(train)

In [99]:
train['pred'] = preds
train_clean = train.drop(train[train['pred']==-1].index, axis=0)
train_clean.drop('pred', axis=1, inplace=True)

In [100]:
mod = LocalOutlierFactor(contamination=0.18, n_neighbors=15, novelty=True)
mod.fit(train_clean)
preds_exo = mod.predict(test.drop('exo', axis=1))

In [101]:
test['pred'] = preds_exo
test['pred'] = np.where(test['pred']==-1, 1, 0)

In [102]:
tn, fp, fn, tp = confusion_matrix(
    test['exo'].values, 
    test['pred'].values,
    ).ravel()

In [103]:
acc = (tp+tn)/(tn+fp+fn+tp)
spec = tn/(tn+fp)
sens = tp/(tp+fn)

print('', acc, spec, sens, sep='\n')


0.6888888888888889
0.6875
0.6904761904761905


---

In [105]:
data = data_raw.drop('exo', axis=1)
data = ((data-data.mean())/data.std())*10
data['exo'] = data_raw['exo']

In [108]:
test = data.loc[:89]
train = data.copy(deep=True)

In [109]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(train.drop('exo', axis=1), train['exo'])

KNeighborsClassifier(n_neighbors=20)

In [110]:
preds = knn.predict(test.drop('exo', axis=1))
true = test['exo'].values

In [111]:
tn, fp, fn, tp = confusion_matrix(
    true, 
    preds,
    ).ravel()

In [112]:
acc = (tp+tn)/(tn+fp+fn+tp)
spec = tn/(tn+fp)
sens = tp/(tp+fn)

print('', acc, spec, sens, sep='\n')


0.5333333333333333
1.0
0.0
